In [ ]:
# Program to generate orders e-mails from Purchasing Portal Active Orders
# Import libraries
import csv
from collections import defaultdict
from datetime import datetime
import os

### TO DO
Write directly to .eml?
Tidy comments

In [ ]:
# Return most recent file in downloads directory
downloads_path = 'C:\\Users\\Purchasing_Kelsey\\Downloads'
most_recent_file = None
most_recent_time = 0

try:
    for entry in os.scandir(downloads_path):
        if entry.is_file():
                # get the modification time of the file using entry.stat().st_mtime_ns
            mod_time = entry.stat().st_mtime_ns
            if mod_time > most_recent_time:
                    # update the most recent file and its modification time
                most_recent_file = entry.name
                most_recent_time = mod_time

except Exception as e:
    print(e)

print(most_recent_file)

In [ ]:
# read most recent file into CSV
data = {}
target_file = downloads_path + '\\' + most_recent_file

try:
    with open (target_file, 'r', encoding='utf-8') as file:
        csvreader = csv.reader(file, delimiter=',')
        fields = next(csvreader)
        for row in csvreader:
            row_dict = dict(zip(fields, row))
            lab_id = row_dict['Lab ID']
            if lab_id in data:
                data[lab_id].append(row_dict)
            else:
                data[lab_id] = [row_dict]
except Exception as e:
    print('Exception:', e)

## TO DO: print the dict

In [ ]:
km_emailed = datetime.today().strftime('%#m/%#d') + ' KM emailed'
print(km_emailed)

In [ ]:
# Clean data

# Remove "orig" from product description
for lab_id, entries in data.items():
    for entry in entries:
        product_description = entry.get('Product Name/Description', '')
        if '(' in product_description:
            product_description = product_description.split('(')[0].strip()
            entry['Product Name/Description'] = product_description

# remove surname from 'Requested by' field
try:
    for lab_id, entries in data.items():
        for entry in entries:
            requested_by = entry.get('Requested by', '')
            if ' ' in requested_by:
                requested_by = requested_by.split(' ')[0].strip()
                entry['Requested by'] = requested_by
except Exception as e:
    print(e)

In [ ]:
# function to write e-mails
def write_order_emails(dict, output_file):
    try:
        with open(output_file, 'w', encoding='utf-8') as file:
            for lab_id, orders in dict.items():
                # generate text for multi-question e-mails
                if len(orders) > 1:
                    # Write the Lab ID and the combined product names for the "Order Questions"
                    file.write(f"Lab ID: {orders[0]['Lab ID']}\n\n")
                    product_names = ' & '.join([order['Product Name/Description'] for order in orders])
                    file.write(f"Order Questions: {product_names}\n\n")
                    
                    # Greeting with the name of the requester from the first order
                    file.write(f"Hello {orders[0]['Requested by']}, \n\n")
                    file.write(f"I have questions about the following orders:\n\n")
                    
                    # Write the details for each order separately
                    for order in orders:               
                        file.write(f"{order['Date requested']}, {order['Vendor Requested']}, ")
                        file.write(f"{order['Product Name/Description']}, {order['Catalog Number Requested']}\n\n")
                        file.write(f"{order['Approval']}\n\n")
                    
                    # Thank you and sign-off once at the end of the email
                    file.write(f"Thanks for your help.\n\n")
                    file.write(f"Best,\n\n")
                    file.write(f"Kelsey\n\n")
                    file.write(km_emailed)
                    file.write(f"\n\n")
                    file.write('-------------------------------------------------\n\n')
                else:
                    # This section handles the case for a single order, as you originally had
                    order = orders[0]
                    file.write(f"Lab ID: {order['Lab ID']}\n\n")
                    file.write(f"Order Question: {order['Product Name/Description']}\n\n")
                    file.write(f"Hello {order['Requested by']},\n\n")
                    file.write(f"I have a question about the following order:\n\n")
                    file.write(f"{order['Date requested']}, {order['Vendor Requested']}, ")
                    file.write(f"{order['Product Name/Description']}, {order['Catalog Number Requested']}\n\n")
                    file.write(f"{order['Approval']}\n\n")
                    file.write(f"Thanks for your help.\n\n")
                    file.write(f"Best,\n\n")
                    file.write(f"Kelsey\n\n")
                    file.write(km_emailed)
                    file.write(f"\n\n")
                    file.write('-------------------------------------------------\n\n')
    except Exception as e:
        print(e)


In [ ]:
# function to export to CSV. Accepts datafranem save destination path, and save file name.
def export_to_csv(df, save_file_path, save_file_name):
    try:
        save_file_name = save_file_name + "_" + datetime.today().strftime('%m.%d.%Y') + ".csv"
        df.to_csv(save_file_path + save_file_name, encoding='utf-8', index=False)
    except Exception as e:
        print(e)
    print(save_file_name, "saved")

In [ ]:
# call function & write emails to .txt
output_name = "OrderQuestions_" + datetime.today().strftime('%#m.%#d.%y') + ".txt"
output_file = "C:\\Users\\Purchasing_Kelsey\\Documents\\K's Projects\\Generate Order Question Emails\\Generated Order Q's\\" + output_name

write_order_emails(data, output_file)
print(output_name, " saved")